In [ ]:
# Copyright 2023 The Cirq Developers
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# State Preparation using Coherent Alias Sampling

Gates for preparing coefficient states.

In section III.D. of the [Linear T paper](https://arxiv.org/abs/1805.03662) the authors introduce
a technique for initializing a state with $L$ unique coefficients (provided by a classical
database) with a number of T gates scaling as 4L + O(log(1/eps)) where eps is the
largest absolute error that one can tolerate in the prepared amplitudes.

In [ ]:
import cirq
import numpy as np
import cirq_ft
import cirq_ft.infra.testing as cq_testing
from cirq_ft.infra.jupyter_tools import display_gate_and_compilation
from typing import *

## `StatePreparationAliasSampling`
Initialize a state with $L$ unique coefficients using coherent alias sampling.

In particular, we take the zero state to:

$$
\sum_{\ell=0}^{L-1} \sqrt{p_\ell} |\ell\rangle |\mathrm{temp}_\ell\rangle
$$

where the probabilities $p_\ell$ are $\mu$-bit binary approximations to the true values and
where the temporary register must be treated with care, see the details in Section III.D. of
the reference.

The preparation is equivalent to [classical alias sampling](https://en.wikipedia.org/wiki/Alias_method):
we sample `l` with probability `p[l]` by first selecting `l` uniformly at random and then
returning it with probability `keep[l] / 2**mu`; otherwise returning `alt[l]`.

Registers:
    selection: The input/output register $|\ell\rangle$ of size lg(L) where the desired
        coefficient state is prepared.
    temp: Work space comprised of sub registers:
        - sigma: A mu-sized register containing uniform probabilities for comparison against
            `keep`.
        - alt: A lg(L)-sized register of alternate indices
        - keep: a mu-sized register of probabilities of keeping the initially sampled index.
        - one bit for the result of the comparison.

This gate corresponds to the following operations:
 - UNIFORM_L on the selection register
 - H^mu on the sigma register
 - QROM addressed by the selection register into the alt and keep registers.
 - LessThanEqualGate comparing the keep and sigma registers.
 - Coherent swap between the selection register and alt register if the comparison
   returns True.

Total space will be (2 * log(L) + 2 mu + 1) work qubits + log(L) ancillas for QROM.
The 1 ancilla in work qubits is for the `LessThanEqualGate` followed by coherent swap.

#### Parameters
 - `lcu_probabilities`: The LCU coefficients.
 - `probability_epsilon`: The desired accuracy to represent each probability (which sets mu size and keep/alt integers). See `openfermion.circuits.lcu_util.preprocess_lcu_coefficients_for_reversible_sampling` for more information. 

#### References
[Encoding Electronic Spectra in Quantum Circuits with Linear T Complexity](https://arxiv.org/abs/1805.03662). Babbush et. al. (2018). Section III.D. and Figure 11.


In [ ]:
coeffs = np.array([1.0, 1, 3, 2])
mu = 3

state_prep = cirq_ft.StatePreparationAliasSampling.from_lcu_probs(
    coeffs, probability_epsilon=2**-mu / len(coeffs)
)
g = cq_testing.GateHelper(
    state_prep
)

display_gate_and_compilation(g)